In [38]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64 # import to define base64 for Graziano Logo


# import correct CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# include correct username and password and CRUD Python module name
username = "aacuser"
password = "SHNU340T"
shelter = AnimalShelter(username, password)


# create dataframe from mongodb query class read method from CRUD class returns cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
# create dash application
app = JupyterDash('Dash AnimalShelter Datatable')

# add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # image file from system
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
   html.Div(id='hidden-div', style={'display':'none'}),
    # format the image size and location and include a URL tag for the client's homepage
   html.A([
        html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image',
                style={"height": "2" "00px","padding":"10px 10px 10px 250px", "align":"center", "margin-left":"auto"
                }),
    ], href='https://www.snhu.edu'),
    # dashboard title
    html.Center(html.B(html.H1('Animal Shelters for Search and Rescue Dashboard'))),
    # unique creator indentifier
    html.Center(html.B(html.H4('Creator: Tiffany Gomez'))),
    # additional user instruction
    html.A(html.P('Select an option to filter dog candidates:')),
    html.Hr(),
    html.Div(
        # Add the interactive filtering options for rescue options and reset
        # Add Radio buttons interactive filter options for data set
        dcc.RadioItems(
            id='filter-options',
            # Required filter options
            options=[
            {'label': ' Water Rescue    ', 'value': 'WTR'},
            {'label': ' Mountain or Wilderness Rescue    ', 'value': 'MT_WLD'},
            {'label': ' Disaster Rescue or Individual Tracking     ', 'value': 'DIS_IND'},
            {'label': ' Reset ', 'value': 'RESET'}
            ],
            value='MTL',                                # default filter options shows none selected
            labelStyle={'display': 'inline-block'}      # filter options positioned in same line
        )


    ),
    html.Hr(),
    # create the data table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],

        # populate the table with records from DataFrame
        data=df.to_dict('records'),
        # set features for interactive data table to make it user-friendly
        editable=False,
        # vertical and horizontal scroll bars
        style_table={'height': 'auto', 'overflowY': 'auto', 'overflowX': 'auto'},
        # format cells to adjust to smaller length of text and limit size of cell for larger text
        style_cell={ 'minWidth': 'auto', 'maxWidth': '150px', 'overflow': 'hidden', 'textOverflow': 'ellipsis'},
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,


        
    ),
    html.Br(),
    html.Hr(),
    # positions pie chart and geolocation chart side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',            # pie chart 
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',              # geolocation chart
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# callbacks for data table and filter options    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-options', 'value')])
def update_dashboard(filter_options):
# MongoDB queries to filter interactive data by calling the read function from the CRUD class
# based on the three filter required by client for sex upon outcome, breed, and age upon outcome
        # Water Rescue option
        if filter_options == 'WTR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesapeake Bay Retriever'},
                                                              {'breed': 'Newfoundland'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        # Mountain/Wilderness option
        elif filter_options == 'MT_WLD':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Siberian Husky'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))
        # Disaster/Individual Tracking option
        elif filter_options == 'DIS_IND':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                })))
        # Resets data table to unfiltered view using MongoDB query to retrive all data again
        elif filter_options == 'RESET':
            df = pd.DataFrame.from_records(shelter.read({}))
                               

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_id': i },
        'background_color': '#ff7373'
    } for i in selected_rows]

# callbacks for current data table to update pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed breeds and counts
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.Plasma, # plotly color scheme
                width=800, 
                height=500   
            )
        )
    ]
# callback for selected datatable entry to update geolocation chart
@app.callback(
     Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
# imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    # Geolocation chart will show the first row of the data table by default if non selected
    if selected_rows == []:
        selected_rows = [0]
    # Geolocation chart will show row of data selected
    if len(selected_rows) == 1:
        return [
            # 13 and 14 represent the longitude and latitude coordinates
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[dff.iloc[selected_rows[0],13], dff.iloc[selected_rows[0],14]], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip for Breed and popup for Animal ID, outcome type, and sex upon outcome
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H3("ID"),
                        html.P(dff.iloc[selected_rows[0],2]),
                        html.H4("Outcome type"),
                        html.P(dff.iloc[selected_rows[0],11]),
                        html.H4("Sex upon outcome type"),
                        html.P(dff.iloc[selected_rows[0],12])
                    ])
                ])
            ])
        ]

app